# Gesture Recognition with Machine Learning 

Welcome to this hands-on machine learning lab! In this exercise, you will build a complete machine learning pipeline to recognize gestures based on IMU (Inertial Measurement Unit) data.

## What You'll Learn
- How to load and explore real sensor data
- Data cleaning and preprocessing techniques
- Normalization and why it matters
- Splitting data for training and validation
- Building and training a neural network with TensorFlow
- Evaluating model performance

## The Data
You'll be working with accelerometer data (X, Y, Z acceleration) collected from an IMU sensor. Each gesture consists of 200 data points (2 seconds of recordings) with labels indicating what gesture was performed.

## Step 1: Import Libraries

First, let's import the libraries we'll need for this lab.

#### Important: If you have include errors but the cell runs you are fine to continue

**Task:** Run the cell below to import the required libraries.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
from tensorflow import keras

print(f"TensorFlow version: {tf.__version__}")
print("Libraries imported successfully!")

## Step 2: Load the Data

Now we need to load our gesture data from the CSV file. The data has the following structure:
- **X-acc, Y-acc, Z-acc**: Acceleration values in three axes
- **label**: The gesture being performed

Each gesture consists of 200 consecutive rows of data.

**Questions to consider:**
1. What function in pandas can read CSV files?
2. How do we specify the separator character?
3. How can we inspect the first few rows?

**Task:** 
1. Load the data from `data/TDATA.csv`
2. Display the first few rows to understand the data structure
3. Check the shape of the dataframe

In [ ]:
# TODO: Load the CSV file
# Hint: Use a pandas function to read the data from the csv file, what separator does it use?
# Your code here

# TODO: Display the first 10 rows
# Hint: DataFrames have a method to show the first N rows - what's it called?
# Your code here

# TODO: Print the shape of the dataframe
# Hint: The shape attribute gives you (rows, columns) as a tuple
# Your code here


## Step 3: Explore and Clean the Data

Before we can use the data, we need to understand it better and clean it.

**Why is data exploration important?**
- We need to know what gestures we have
- Missing data can break our model
- Imbalanced data (too much of one gesture, too little of another) can cause bias

**Questions to investigate:**
1. How many unique gestures are in the dataset?
2. Are there any missing values?
3. What is the distribution of different gestures?
4. Do we have balanced data (similar amounts of each gesture)?

**Task:** The visualization code is provided below. Fill in the data exploration code.

In [ ]:
# TODO: Find unique gesture labels
# Hint: Column data has a unique() method that returns all distinct values, Column indexing in Pandas is done using data['col_name']
# Your code here

# TODO: Check for missing values
# Hint: Chain isnull() to find missing values and sum() to count them per column
# Your code here

# TODO: Count how many samples of each gesture we have
# Hint: The value_counts() method shows how many times each unique value appears
# Your code here

# Visualization code provided - no need to change this
plt.figure(figsize=(10, 6))
gesture_counts.plot(kind='bar', color='skyblue', edgecolor='black')
plt.xlabel('Gesture')
plt.ylabel('Number of Samples (rows)')
plt.title('Distribution of Gesture Samples')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()


## Step 4: Preprocess the Data

Since each gesture consists of 200 consecutive data points, we need to reshape our data accordingly.

**Understanding the data structure:**
- Currently: One long dataframe where every 200 rows = 1 gesture
- Goal: Separate 3D array where each gesture is its own unit

**What we're creating:**
- **X**: An array of shape (num_gestures, 200, 3) containing the acceleration data
  - Think of it as: [gesture1[200 samples[x, y, z]], gesture2[...], ...]
- **y**: An array of shape (num_gestures,) containing the labels
  - One label per gesture

**Questions to consider:**
1. How do we ensure we only use complete gestures (no partial data)?
2. How does numpy's reshape work with multi-dimensional arrays?
3. Why do we only need every 200th label?

**Task:** Complete the preprocessing steps to reshape the data.

In [ ]:
# TODO: Remove rows with missing values (if any)
# Hint: There's a DataFrame method to drop rows with NaN values
# Your code here

# Define the samples per gesture (this is given)
SAMPLES_PER_GESTURE = 200

# TODO: Calculate the number of complete gestures
# Hint: Use integer division (//) to find how many complete 200-sample blocks fit in the data
# Your code here

# TODO: Keep only the rows that form complete gestures
# Hint: Use iloc to slice - multiply num_gestures by SAMPLES_PER_GESTURE to get the cutoff
# Your code here

# TODO: Extract the acceleration values (X-acc, Y-acc, Z-acc)
# Hint: Select multiple columns using a list, then convert to numpy array with .values
# Your code here

# TODO: Reshape into the correct form
# Hint: The reshape method takes the new dimensions, think about how many gestures, samples per gesture, and axes you have
# Your code here

# TODO: Extract labels (take every 200th label since they're the same for each gesture)
# Hint: Use iloc with step slicing [::step] to skip rows - what step gets every 200th row?
# Your code here

print(f"\nFinal shapes:")
print(f"X shape: {X.shape} - ({num_gestures} gestures, {SAMPLES_PER_GESTURE} samples each, 3 axes)")
print(f"y shape: {y.shape} - ({num_gestures} labels)")

## Step 5: Visualize Sample Gestures

Let's visualize what a gesture looks like! This helps us understand the data better.

**What to look for in the plots:**
- Do the three axes (X, Y, Z) show different patterns?
- Can you see distinct patterns for different gestures?
- Are there any obvious anomalies or noise?

**Note:** Visualization code is provided - just run it to see the results!

In [ ]:
# Visualization code provided - just run this cell!

# Pick a sample gesture to visualize
sample_idx = 60

# Create a plot with the three acceleration axes
plt.figure(figsize=(12, 4))
plt.plot(X[sample_idx, :, 0], label='X-axis', alpha=0.7, linewidth=1.5)
plt.plot(X[sample_idx, :, 1], label='Y-axis', alpha=0.7, linewidth=1.5)
plt.plot(X[sample_idx, :, 2], label='Z-axis', alpha=0.7, linewidth=1.5)
plt.xlabel('Sample')
plt.ylabel('Acceleration')
plt.title(f'Gesture: {y.iloc[sample_idx]}')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Visualize multiple gestures for comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 8))
axes = axes.flatten()

for i in range(min(4, num_gestures)):
    axes[i].plot(X[i+sample_idx, :, 0], label='X-axis', alpha=0.7)
    axes[i].plot(X[i+sample_idx, :, 1], label='Y-axis', alpha=0.7)
    axes[i].plot(X[i+sample_idx, :, 2], label='Z-axis', alpha=0.7)
    axes[i].set_xlabel('Sample')
    axes[i].set_ylabel('Acceleration')
    axes[i].set_title(f'Gesture {i+sample_idx}: {y.iloc[i+sample_idx]}')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Step 6: Encode Labels

Neural networks work with numbers, not text labels. We need to convert our gesture labels into numerical values.

**Why do we need label encoding?**
- Machine learning models perform mathematical operations
- We can't do math on text strings like "idle" or "waving"
- LabelEncoder converts: "idle" → 0, "sliding" → 1, "waving" → 2

**Questions to consider:**
1. Why does the specific number assigned to each label not matter?
2. What would happen if we manually assigned numbers inconsistently?

**Task:** Use LabelEncoder to convert text labels to numbers.

In [ ]:
# TODO: Create a LabelEncoder
# Hint: You need to instantiate the LabelEncoder class (create an object from it)
# Your code here

# TODO: Fit and transform the labels
# Hint: LabelEncoder has a fit_transform() method that learns the labels and converts them in one step
# Your code here

# Display the mapping (code provided)
print("Label mapping:")
print("="*40)
for i, label in enumerate(label_encoder.classes_):
    print(f"{label:20} -> {i}")

print(f"\nEncoded labels shape: {y_encoded.shape}")
print(f"Encoded labels sample (first 10): {y_encoded[:10]}")


## Step 7: Normalize the Data

Neural networks train better when the input features are on a similar scale. We'll use standardization (zero mean, unit variance).

**Why normalize?**
- Different sensors might have different ranges (e.g., X: -100 to 100, Y: -10 to 10)
- Large values can dominate the learning process
- Standardization puts all features on the same scale
- Formula: `z = (x - mean) / std_deviation`

**What is StandardScaler doing?**
- Calculates mean and standard deviation for each feature
- Subtracts mean (centers data at 0)
- Divides by standard deviation (scales to unit variance)

**Questions to consider:**
1. Why do we flatten before scaling and reshape after?
2. What does "zero mean, unit variance" mean for the data?

**Task:** Apply StandardScaler to normalize the acceleration data.

In [ ]:
# TODO: Reshape X to 2D for scaling
# Hint: StandardScaler expects 2D input. Use -1 in reshape to automatically calculate that dimension
# We then need to flatten from (num_gestures, 200, 3) to (x,y), what does x and y need to be?
# Your code here

# TODO: Create and fit the scaler
# Hint: Instantiate StandardScaler, similar to how you created LabelEncoder
# Your code here

# TODO: Transform the data
# Hint: there is a function that learns the mean/std and applies the transformation at once
# Your code here

# TODO: Reshape back to 3D
# Hint: We need to restore the original 3D structure (num_gestures, SAMPLES_PER_GESTURE, 3) for the normalized data
# Your code here

print(f"\nNormalized data shape: {X_normalized.shape}")
print(f"Mean (should be close to 0): {X_normalized.mean():.6f}")
print(f"Std (should be close to 1): {X_normalized.std():.6f}")

# Show statistics per axis
print(f"\nPer-axis statistics:")
for i, axis_name in enumerate(['X', 'Y', 'Z']):
    print(f"{axis_name}-axis - Mean: {X_normalized[:, :, i].mean():.6f}, Std: {X_normalized[:, :, i].std():.6f}")


## Step 8: Data Augmentation

More data often leads to better models! We can create additional training samples by augmenting our existing data.

**Why augment time-series data?**
- We only have 150 gestures - not a lot for training a neural network
- Many gestures look similar when played in reverse
- By flipping the time axis, we can effectively double our dataset

**How does time reversal work?**
- Original: samples [0, 1, 2, ..., 198, 199]
- Reversed: samples [199, 198, 197, ..., 1, 0]
- The gesture played backward is still a valid gesture pattern

**Questions to consider:**
1. Why does time reversal make sense for gesture data but maybe not for other time series?
2. What other augmentation techniques could work for IMU data?

**Task:** Create time-reversed versions and combine with original data.

In [ ]:
# TODO: Create augmented data by reversing the time axis
# Hint: In shape (num_gestures, 200, 3), axis=1 is the time dimension
# Array slicing [:, ::-1, :] reverses the middle dimension - what does ::-1 mean?
# Your code here

print(f"Original data shape: {X_normalized.shape}")
print(f"Augmented data shape: {X_augmented.shape}")

# TODO: Combine original and augmented data
# Hint: there is a function that stacks arrays vertically (adds more rows)
# Your code here
# Hint: np.concatenate() joins arrays we want to append the labels again for the new gestures
# Your code here

print(f"\nCombined data shape: {X_combined.shape}")
print(f"Combined labels shape: {y_combined.shape}")
print(f"Dataset size increased from {num_gestures} to {len(X_combined)} samples")


## Step 9: Visualize Original vs Augmented

Let's visualize the effect of our data augmentation to understand what we've done.

**What to observe:**
- The augmented gesture should be a mirror image along the time axis
- Both should represent valid movement patterns

**Note:** Visualization code is provided - just run it!

In [ ]:
# Visualization code provided - just run this cell!

# TODO: Visualize original vs augmented gesture
# Think about: How can you compare the original and time-reversed versions side by side?
sample_idx = 60

plt.figure(figsize=(14, 4))

# Original gesture
plt.subplot(1, 2, 1)
plt.plot(X_normalized[sample_idx, :, 0], label='X-axis', alpha=0.7, linewidth=1.5)
plt.plot(X_normalized[sample_idx, :, 1], label='Y-axis', alpha=0.7, linewidth=1.5)
plt.plot(X_normalized[sample_idx, :, 2], label='Z-axis', alpha=0.7, linewidth=1.5)
plt.xlabel('Sample')
plt.ylabel('Normalized Acceleration')
plt.title(f'Original Gesture: {label_encoder.inverse_transform([y_encoded[sample_idx]])[0]}')
plt.legend()
plt.grid(True, alpha=0.3)

# Augmented gesture (time-reversed)
plt.subplot(1, 2, 2)
plt.plot(X_augmented[sample_idx, :, 0], label='X-axis', alpha=0.7, linewidth=1.5)
plt.plot(X_augmented[sample_idx, :, 1], label='Y-axis', alpha=0.7, linewidth=1.5)
plt.plot(X_augmented[sample_idx, :, 2], label='Z-axis', alpha=0.7, linewidth=1.5)
plt.xlabel('Sample')
plt.ylabel('Normalized Acceleration')
plt.title(f'Augmented (Time-Reversed): {label_encoder.inverse_transform([y_encoded[sample_idx]])[0]}')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Notice how the augmented gesture is the mirror image of the original along the time axis!")

## Step 10: Split the Data

We need to split our data into training and testing sets.

**Why split the data?**
- **Training set (80%)**: The model learns from this data
- **Testing set (20%)**: We evaluate on this to see if the model generalizes
- If we test on training data, we can't tell if the model truly learned or just memorized

**What is stratification?**
- Ensures both train and test sets have similar proportions of each gesture
- Example: If we have 33% idle gestures, both sets will have ~33% idle
- Prevents bias from imbalanced splits

**Questions to consider:**
1. Why is 80/20 a common split ratio?
2. What would happen if we used 50/50 instead?
3. Why set a random_state?

**Task:** Split the combined augmented data with 80% for training and 20% for testing.

In [ ]:
# TODO: Split the combined data with 80/20 ratio
# Hint: train_test_split separates your data into training and testing sets
# What parameter controls the split ratio? (test_size=?)
# What parameter ensures balanced classes in both sets? (stratify=?)
# Your code here

print(f"Training samples: {X_train.shape[0]}")
print(f"Testing samples: {X_test.shape[0]}")
print(f"\nTraining set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")

# Show class distribution (code provided)
print(f"\nTraining set class distribution:")
unique, counts = np.unique(y_train, return_counts=True)
for label_idx, count in zip(unique, counts):
    label_name = label_encoder.inverse_transform([label_idx])[0]
    print(f"{label_name:20} : {count} samples")

print(f"\nTesting set class distribution:")
unique, counts = np.unique(y_test, return_counts=True)
for label_idx, count in zip(unique, counts):
    label_name = label_encoder.inverse_transform([label_idx])[0]
    print(f"{label_name:20} : {count} samples")


## Step 11: Build the Neural Network

Now for the exciting part - building the model! We'll use a simple neural network architecture:
1. **Flatten layer:** Converts 3D input to 1D
2. **Dense layers:** Fully connected layers that learn patterns
3. **Dropout:** Prevents overfitting
4. **Output layer:** Produces predictions for each gesture class

**Questions to answer (check answers.py):**
1. What layer type converts 3D input (200, 3) to 1D (600)?
2. What activation function is commonly used in hidden layers for non-linearity?
3. What activation function should the output layer use for multi-class classification?
4. What does the Dropout layer do to help the model?
5. In Dense(128, activation='relu'), what does the 128 parameter specify?

**Task:** Complete the model architecture by filling in the missing parts.

In [ ]:
# Determine the number of classes
num_classes = len(label_encoder.classes_)
print(f"Number of gesture classes: {num_classes}")

# TODO: Build the Sequential model
# Hint: You'll need to create an empty Sequential model, then add layers to it one by one
# Your code here

# TODO: Add Input layer first
# Hint: Use keras.layers.Input with shape parameter to define the input shape
# The shape should be (SAMPLES_PER_GESTURE, 3) for 200 timesteps and 3 axes
# This is the recommended way to specify input shape in modern Keras
# Your code here

# TODO: Add Flatten layer
# Hint: This layer converts 3D data to 1D (from (200, 3) to 600 features)
# Why? Dense layers need 1D input, but our data is (200 samples, 3 axes)
# Since we already added Input layer, we don't need to specify input_shape here
# Your code here

# TODO: Add first Dense layer
# Hint: We need 128 neurons with ReLU activation for the first hidden layer
# Why 128? It provides enough learning capacity without being too large but it is an arbitrary choice
# Your code here

# TODO: Add first Dropout layer
# Hint: Dropout randomly turns off a percentage of neurons during training to prevent overfitting
# The rate parameter controls what percentage gets dropped
# Your code here

# TODO: Add second Dense layer
# Hint: 64 neurons with ReLU activation - gradually reducing layer size is common
# Your code here

# TODO: Add second Dropout layer
# Hint: Another 30% dropout for regularization
# Your code here

# TODO: Add output Dense layer
# Hint: Output layer needs as many neurons as we have classes
# What activation function converts outputs to probabilities that sum to 1?
# Your code here

# Display the model architecture (code provided)
print("\nModel Architecture:")
print("="*70)
model.summary()

## Step 12: Compile the Model

Before training, we need to configure the learning process:
- **Optimizer:** Algorithm to update weights during training (Adam is adaptive and works well)
- **Loss function:** Measures how wrong the predictions are
- **Metrics:** What we want to track during training (accuracy)

**Questions to answer (check answers.py):**
1. What optimizer are we using in this lab?
2. What loss function should be used for multi-class classification with integer labels?

**Why these choices?**
- **Adam:** Adaptive learning rate optimizer that works well in most cases
- **Sparse Categorical Crossentropy:** For multi-class classification when labels are integers (0, 1, 2) not one-hot encoded
- **Accuracy:** Easy to interpret metric (percentage of correct predictions)

**Task:** Compile the model with the appropriate optimizer, loss function, and metrics.

In [ ]:
# TODO: Compile the model
# Hint: You need an optimizer (adam works well), a loss function for multi-class classification, and metrics to track
# Why these choices?
# Your code here

print("Model compiled successfully!")
print(f"\nOptimizer: Adam")
print(f"Loss function: Sparse Categorical Crossentropy")
print(f"Metrics: Accuracy")


## Step 13: Train the Model

Time to train! The model will learn patterns from the training data.

**Understanding the training parameters:**
- **Epochs:** Number of times to go through the entire training dataset
  - More epochs = more learning, but risk of overfitting
  - We'll use 50 epochs
- **Batch size:** Number of samples processed before updating weights
  - Smaller batches = more frequent updates, noisier gradient
  - Larger batches = more stable gradient, requires more memory
  - We'll use 32 (a common choice)
- **Validation split:** Portion of training data reserved for validation
  - Used to monitor performance during training
  - We'll use 0.2 (20% of training data)

**Task:** Train the model with the specified parameters.

In [ ]:
# TODO: Train the model
# Hint: The fit() method trains the model. You need to specify:
# - How many epochs (complete passes through the data)
# - Batch size (how many samples to process before updating weights)
# - Validation split (what fraction of training data to use for validation)
# Your code here

print("\nTraining complete!")


## Step 14: Visualize Training History

Let's see how the model improved during training by plotting the training curves.

**What to look for:**
- **Training vs Validation Accuracy:** Should both increase over time
  - If validation accuracy is much lower than training: overfitting
  - If they're similar: model is generalizing well
- **Training vs Validation Loss:** Should both decrease over time
  - If validation loss increases while training loss decreases: overfitting

**Questions to think about:**
1. How do you access the training accuracy from the history object?
2. What does it mean if validation loss starts increasing while training loss keeps decreasing?
3. At what epoch does the model seem to converge (stop improving)?

**Task:** The plotting code is provided. Run it and interpret the results!

In [ ]:
# Plotting code provided - just run this cell!
# Hint: The history object stores training metrics for each epoch
# Access them via: history.history['accuracy'], history.history['val_accuracy'], etc.

plt.figure(figsize=(14, 5))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.title('Model Accuracy', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss', linewidth=2)
plt.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Model Loss', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print final training metrics
print(f"Final Training Accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}")
print(f"Final Training Loss: {history.history['loss'][-1]:.4f}")
print(f"Final Validation Loss: {history.history['val_loss'][-1]:.4f}")

print("\nThink about:")
print("- Is there a gap between training and validation accuracy?")
print("- Does the model show signs of overfitting?")
print("- Could we have stopped training earlier?")


## Step 15: Evaluate on Test Set

Now let's see how well the model performs on completely unseen data!

**Why evaluate on test data?**
- Training accuracy can be misleading (the model has seen this data)
- Validation accuracy is better but still used during training decisions
- **Test accuracy** is the true measure of how well the model generalizes

**Questions to think about:**
1. How does test accuracy compare to training accuracy?
2. If test accuracy is much lower, what might that indicate?
3. What does the test loss value tell us?

**Task:** Evaluate the model on the test set using model.evaluate().

In [ ]:
# TODO: Evaluate the model on the test set
# Hint: The evaluate() method returns loss and accuracy on unseen data
# Pass the test data (X_test, y_test) to see how well the model generalizes
# Your code here

print(f"\n{'='*60}")
print(f"Test Set Performance")
print(f"{'='*60}")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")
print(f"{'='*60}")

print("\nCompare this to your training/validation results:")
print("- Is test accuracy similar to validation accuracy?")
print("- If yes: Great! The model generalizes well")
print("- If no: The model may be overfitting or underfitting")


## Step 16: Make Predictions

Let's use the model to predict gestures on individual test samples!

**Understanding predictions:**
- **model.predict()** returns probabilities for each class
- Shape: (num_samples, num_classes) - e.g., (60, 3)
- Each row sums to 1.0 (due to softmax activation)
- **np.argmax()** finds the class with highest probability

**Example prediction output:**
```
[0.05, 0.90, 0.05] → Class 1 (90% confidence)
```

**Questions to think about:**
1. What does model.predict() return?
2. Why do we use np.argmax() on the predictions?
3. What does the confidence percentage represent?

**Task:** Make predictions on the test set and analyze the results.

In [ ]:
# TODO: Make predictions on the test set
# Hint: The predict() method returns probabilities for each class
# Shape will be (num_samples, num_classes) - one probability distribution per sample
# Your code here

# TODO: Convert probabilities to class labels
# Hint: Predictions are probabilities for each class. Use argmax to find which class has highest probability
# Remember: axis=1 means "find max along each row"
# Your code here

# Display some predictions (code provided)
print("Sample Predictions:")
print("="*60)
for i in range(min(20, len(y_test))):
    actual = label_encoder.inverse_transform([y_test[i]])[0]
    predicted = label_encoder.inverse_transform([predicted_classes[i]])[0]
    confidence = predictions[i][predicted_classes[i]] * 100
    correct = "✓" if actual == predicted else "✗"
    print(f"{correct} Actual: {actual:15} | Predicted: {predicted:15} ({confidence:.1f}%)")

# Calculate accuracy (code provided)
correct_predictions = np.sum(predicted_classes == y_test)
total_predictions = len(y_test)
accuracy = correct_predictions / total_predictions * 100

print(f"\n{'='*60}")
print(f"Correct predictions: {correct_predictions}/{total_predictions} ({accuracy:.2f}%)")
print(f"{'='*60}")

print("\nNotice:")
print("- High confidence (>90%) usually means the model is certain")
print("- Low confidence (<60%) might indicate uncertainty or ambiguous samples")
print("- Check if misclassifications have lower confidence")


## Step 17: Confusion Matrix

A confusion matrix shows which gestures the model confuses with each other.

**Understanding the confusion matrix:**
- Rows: Actual labels
- Columns: Predicted labels
- Diagonal: Correct predictions
- Off-diagonal: Misclassifications

**What to look for:**
- Which gestures are confused with each other?
- Are certain gestures harder to classify?
- Is the confusion symmetric (A→B vs B→A)?

**Questions to think about:**
1. How do you create a confusion matrix from actual and predicted labels?
2. What would a perfect confusion matrix look like?
3. If "waving" is often confused with "sliding", what might that tell us about the data?

**Task:** Create and visualize the confusion matrix.

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# TODO: Compute the confusion matrix
# Hint: confusion_matrix compares actual labels (y_test) with predicted labels
# Which order should they go in? (actual, predicted) or (predicted, actual)?
# Your code here

# Visualization code provided - just run it!
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_,
            cbar_kws={'label': 'Count'})
plt.xlabel('Predicted', fontsize=12, fontweight='bold')
plt.ylabel('Actual', fontsize=12, fontweight='bold')
plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Print classification report (code provided)
print("\nClassification Report:")
print("="*70)
print(classification_report(y_test, predicted_classes,
                           target_names=label_encoder.classes_,
                           digits=4))

print("\nInterpret the confusion matrix:")
print("- Perfect predictions would have all values on the diagonal")
print("- Off-diagonal values show which gestures are confused")
print("- Use this to understand where the model struggles")


## Step 18: Experiment and Improve

Congratulations on completing the basic pipeline! Now it's time to experiment and see if you can improve the model.

**Experimentation ideas:**

1. **Architecture changes:**
   - Try different numbers of neurons (e.g., 256, 512)
   - Add more layers or remove layers
   - Try different dropout rates (0.2, 0.4, 0.5)

2. **Training changes:**
   - Increase/decrease epochs
   - Try different batch sizes (16, 64, 128)
   - Try different optimizers ('sgd', 'rmsprop')

3. **Advanced techniques:**
   - Add batch normalization layers
   - Try different activation functions
   - Implement early stopping

**Questions to explore:**
- Does a deeper network always perform better?
- What happens with very high dropout (0.7)?
- How does the model perform with fewer epochs?

**Task:** Build an alternative model and compare its performance!

In [ ]:
# Example: Building a deeper model
model_v2 = keras.Sequential([
    keras.layers.Input(shape=(SAMPLES_PER_GESTURE, 3)),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(num_classes, activation='softmax')
])

model_v2.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("Alternative Model Architecture:")
model_v2.summary()

#Uncomment to train the alternative model
history_v2 = model_v2.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

## Summary

In this lab, you've completed a full machine learning workflow:

1. ✓ **Data Collection**: Loaded IMU sensor data
2. ✓ **Data Cleaning**: Removed invalid entries
3. ✓ **Preprocessing**: Reshaped data into gestures
4. ✓ **Normalization**: Standardized features for better training
5. ✓ **Data Splitting**: Created train/test sets
6. ✓ **Model Building**: Designed a neural network
7. ✓ **Training**: Taught the model to recognize gestures
8. ✓ **Validation**: Evaluated performance on unseen data

Great work! You now have hands-on experience with the complete machine learning pipeline.

---

